# 🤖 Gemma-2-2b-it 分類モデル訓練・保存ノートブック

## 概要
このノートブックは、MAP - Charting Student Math Misunderstandingsコンペティション用に、Gemma-2-2b-itモデルを6分類タスクでファインチューニングし、保存することを目的としています。

### モデル仕様
- **ベースモデル**: google/gemma-2-2b-it (~2.6B parameters)
- **タスク**: 6クラス分類 (True/False_Correct/Neither/Misconception)
- **ファインチューニング**: LoRA (Low-Rank Adaptation)
- **GPU**: A100 推奨

### 保存形式
- Google Drive: `/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/`
- ローカル: zipファイルでダウンロード
- 形式: Kaggle提出用のAutoModelForSequenceClassification

## 📦 環境セットアップ

In [ ]:
# Google Driveマウント
from google.colab import drive
drive.mount('/content/drive')

# 必要なディレクトリ作成
import os
os.makedirs('/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models', exist_ok=True)
os.makedirs('/content/drive/MyDrive/kaggleCompe_MAP-math/map_data', exist_ok=True)
os.makedirs('/content/drive/MyDrive/kaggleCompe_MAP-math/logs', exist_ok=True)

print("✅ Google Drive マウント完了")
print("✅ 必要ディレクトリ作成完了")

In [ ]:
# Hugging Face認証
!huggingface-cli login

In [ ]:
# 必要ライブラリのインストール
!pip install peft accelerate transformers datasets -q
print("✅ ライブラリインストール完了")

## 📚 ライブラリとモデル設定

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from peft import LoraConfig, get_peft_model
import warnings
import json
import shutil
from pathlib import Path

warnings.filterwarnings("ignore")

# モデル設定
MODEL_NAME = "google/gemma-2-2b-it"
NUM_LABELS = 6
MAX_LENGTH = 512

# デバイス設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ 使用デバイス: {device}")

if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("✅ ライブラリ読み込み完了")

## 📊 データ読み込みと前処理

In [ ]:
def load_and_prepare_data():
    """データの読み込みと前処理（6分類形式）"""
    print("=" * 60)
    print("📊 データ読み込みと前処理（6分類形式）")
    print("=" * 60)

    # データパス（Google Drive内）
    train_data_path = "/content/drive/MyDrive/kaggleCompe_MAP-math/map_data/train.csv"
    
    if not os.path.exists(train_data_path):
        print(f"❌ データが見つかりません: {train_data_path}")
        print("💡 Google DriveのMyDrive/kaggleCompe_MAP-math/map_data/に train.csv をアップロードしてください")
        return None

    try:
        # データ読み込み
        train_df = pd.read_csv(train_data_path)
        print(f"✅ 訓練データ読み込み: {train_df.shape}")

        # コンペ形式のターゲット確認（6分類）
        print("\n📋 Category分布:")
        category_counts = train_df["Category"].value_counts()
        print(category_counts)

        # NaN値除去
        before_len = len(train_df)
        train_df = train_df.dropna(subset=["Category", "StudentExplanation"])
        after_len = len(train_df)
        print(f"\n🧹 NaN除去: {before_len} -> {after_len} ({before_len - after_len}行削除)")

        # 強化テキスト特徴量作成
        def create_enhanced_text(row):
            """Question + MC_Answer + Student Explanation を統合"""
            question = str(row["QuestionText"]) if pd.notna(row["QuestionText"]) else ""
            mc_answer = str(row["MC_Answer"]) if pd.notna(row["MC_Answer"]) else ""
            explanation = str(row["StudentExplanation"]) if pd.notna(row["StudentExplanation"]) else ""
            
            enhanced_text = f"Question: {question} Selected Answer: {mc_answer} Explanation: {explanation}"
            return enhanced_text

        print("\n🔧 強化テキスト特徴量作成中...")
        train_df["enhanced_text"] = train_df.apply(create_enhanced_text, axis=1)

        # 6つのカテゴリの確認
        unique_categories = sorted(train_df["Category"].unique())
        print(f"\n📊 ユニークなカテゴリ数: {len(unique_categories)}")
        print("カテゴリ一覧:")
        for i, cat in enumerate(unique_categories):
            count = (train_df["Category"] == cat).sum()
            percentage = count / len(train_df) * 100
            print(f"  {i}: {cat} ({count:,}件, {percentage:.1f}%)")

        # テキスト長統計
        text_lengths = train_df["enhanced_text"].str.len()
        print(f"\n📝 テキスト長統計:")
        print(f"  平均: {text_lengths.mean():.1f} 文字")
        print(f"  中央値: {text_lengths.median():.1f} 文字")
        print(f"  最大: {text_lengths.max()} 文字")
        print(f"  512文字以下: {(text_lengths <= 512).sum()} ({(text_lengths <= 512).mean()*100:.1f}%)")

        return train_df

    except Exception as e:
        print(f"❌ データ読み込みまたは前処理に失敗: {e}")
        return None

# データ読み込み実行
train_df = load_and_prepare_data()

## 🤖 Gemmaモデルとトークナイザー読み込み

In [ ]:
def load_gemma_model(device):
    """Gemma-2-2b-itモデルとトークナイザーの読み込み（元の定義に準拠）"""
    print("\n" + "=" * 60)
    print(f"🤖 Gemmaモデル読み込み: {MODEL_NAME}")
    print("=" * 60)

    try:
        print("📝 Gemmaトークナイザー読み込み中...")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

        # パディングトークンの設定（Gemmaの場合必要）
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            print("🔧 パディングトークンをEOSトークンに設定")

        print(f"✅ トークナイザー読み込み完了")
        print(f"🔖 パディングトークン: {tokenizer.pad_token}")
        print(f"📏 語彙サイズ: {tokenizer.vocab_size:,}")

        print("\n🧠 Gemmaモデル読み込み中...")

        # 分類タスク用の設定
        config = AutoConfig.from_pretrained(MODEL_NAME)
        config.num_labels = NUM_LABELS
        config.problem_type = "single_label_classification"

        # Gemma-2-2b-itモデルを分類用に読み込み
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_NAME,
            config=config,
            torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
            device_map="auto" if device.type == "cuda" else None,
        )

        print(f"✅ Gemmaモデル読み込み完了!")

        # モデル情報表示
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

        print(f"📊 分類クラス数: {NUM_LABELS}")
        print(f"📈 総パラメータ数: {total_params:,}")
        print(f"🎯 訓練可能パラメータ数: {trainable_params:,}")
        print(f"💡 モデルサイズ: ~{total_params / 1e9:.2f}B parameters")

        return model, tokenizer

    except Exception as e:
        print(f"❌ モデル読み込み失敗: {e}")
        return None, None

# モデル読み込み実行
model, tokenizer = load_gemma_model(device)

## 🔧 LoRA設定とPEFT適用

In [ ]:
def apply_lora_to_model(model):
    """LoRA設定とPEFT適用"""
    print("\n" + "=" * 60)
    print("🔧 LoRA (PEFT) 設定と適用")
    print("=" * 60)

    try:
        # LoRA設定
        lora_config = LoraConfig(
            r=16,                    # LoRA attention dimension
            lora_alpha=32,          # Alpha parameter for LoRA scaling
            target_modules=[        # Gemma-2の主要な線形層をターゲット
                "q_proj", "k_proj", "v_proj", "o_proj",
                "gate_proj", "up_proj", "down_proj"
            ],
            lora_dropout=0.05,      # Dropout probability for LoRA layers
            bias="none",            # Bias type
            task_type="SEQ_CLS",    # Sequence Classification task
            modules_to_save=["classifier", "score"],  # 分類ヘッドを保存
        )

        print("📋 LoRA設定:")
        print(f"  r (rank): {lora_config.r}")
        print(f"  lora_alpha: {lora_config.lora_alpha}")
        print(f"  target_modules: {lora_config.target_modules}")
        print(f"  lora_dropout: {lora_config.lora_dropout}")
        print(f"  task_type: {lora_config.task_type}")

        # PEFTモデル作成
        print("\n🔄 PEFTモデル作成中...")
        peft_model = get_peft_model(model, lora_config)
        
        print("✅ PEFTモデル作成完了")
        peft_model.print_trainable_parameters()

        # デバイスに移動
        peft_model.to(device)
        print(f"📍 PEFTモデルをデバイスに移動: {next(peft_model.parameters()).device}")

        return peft_model

    except Exception as e:
        print(f"❌ LoRA適用失敗: {e}")
        return None

# LoRA適用
if model is not None:
    peft_model = apply_lora_to_model(model)
else:
    print("❌ モデルが読み込まれていないため、LoRAを適用できません")
    peft_model = None

## 📝 データセットクラス定義

In [ ]:
class MathMisconceptionDataset(Dataset):
    """Math Misconception Dataset for PyTorch"""

    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # トークン化
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long),
        }

print("✅ MathMisconceptionDataset クラス定義完了")

## 📊 評価メトリクス定義

In [ ]:
def compute_map3_metrics(eval_pred):
    """MAP@3メトリクスの計算"""
    predictions, labels = eval_pred
    predictions = torch.softmax(torch.tensor(predictions), dim=-1).numpy()

    map_scores = []
    for i, true_label in enumerate(labels):
        top3_indices = np.argsort(predictions[i])[::-1][:3]
        score = 0.0
        for j, pred_idx in enumerate(top3_indices):
            if pred_idx == true_label:
                score = 1.0 / (j + 1)
                break
        map_scores.append(score)

    map3_score = np.mean(map_scores)
    accuracy = accuracy_score(labels, np.argmax(predictions, axis=1))

    return {"map3": map3_score, "accuracy": accuracy}

print("✅ MAP@3評価メトリクス定義完了")

## 🚀 モデル訓練実行

In [ ]:
def train_model(train_df, peft_model, tokenizer, device):
    """モデルのファインチューニング実行"""
    print("\n" + "=" * 60)
    print("🚀 6分類モデル ファインチューニング開始")
    print("=" * 60)

    if train_df is None or peft_model is None:
        print("❌ 訓練データまたはモデルが準備されていません")
        return None, None

    # ラベルエンコーディング
    label_encoder = LabelEncoder()
    train_df["encoded_labels"] = label_encoder.fit_transform(train_df["Category"])

    print(f"📋 エンコードされたラベル:")
    for i, label in enumerate(label_encoder.classes_):
        count = (train_df["encoded_labels"] == i).sum()
        print(f"  {i}: {label} ({count:,}件)")

    # 訓練/検証分割
    train_texts = train_df["enhanced_text"].tolist()
    train_labels = train_df["encoded_labels"].tolist()

    try:
        X_train, X_val, y_train, y_val = train_test_split(
            train_texts,
            train_labels,
            test_size=0.2,
            random_state=42,
            stratify=train_labels,
        )
        print("✅ Stratified split適用")
    except ValueError:
        X_train, X_val, y_train, y_val = train_test_split(
            train_texts, train_labels, test_size=0.2, random_state=42
        )
        print("✅ Regular split適用（少数クラスのため）")

    print(f"📊 訓練データ: {len(X_train):,}件")
    print(f"📊 検証データ: {len(X_val):,}件")

    # データセット作成
    train_dataset = MathMisconceptionDataset(
        X_train, y_train, tokenizer, max_length=MAX_LENGTH
    )
    val_dataset = MathMisconceptionDataset(
        X_val, y_val, tokenizer, max_length=MAX_LENGTH
    )

    # 訓練設定
    training_args = TrainingArguments(
        output_dir="/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/gemma-2-2b-math-classification",
        num_train_epochs=3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=8,  # 実効バッチサイズ32
        warmup_steps=200,
        weight_decay=0.01,
        logging_dir="/content/drive/MyDrive/kaggleCompe_MAP-math/logs",
        logging_steps=100,
        eval_strategy="steps",
        eval_steps=300,
        save_strategy="steps",
        save_steps=300,
        load_best_model_at_end=True,
        metric_for_best_model="map3",
        greater_is_better=True,
        report_to="none",
        dataloader_pin_memory=False,
        fp16=False,
        bf16=True,
        optim="adamw_torch",
        learning_rate=2e-5,
        save_total_limit=2,
        gradient_checkpointing=False,
    )

    print("\n📋 訓練設定:")
    print(f"  エポック数: {training_args.num_train_epochs}")
    print(f"  バッチサイズ: {training_args.per_device_train_batch_size}")
    print(f"  勾配累積: {training_args.gradient_accumulation_steps}")
    print(f"  学習率: {training_args.learning_rate}")
    print(f"  保存先: {training_args.output_dir}")

    # Trainer作成
    trainer = Trainer(
        model=peft_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_map3_metrics,
    )

    # 訓練実行
    print("\n🚀 ファインチューニング開始...")
    try:
        trainer.train()
        print("✅ ファインチューニング完了")

        # 最終評価
        print("\n📊 最終評価:")
        eval_results = trainer.evaluate()
        for key, value in eval_results.items():
            print(f"  {key}: {value:.4f}")

        return trainer, label_encoder

    except Exception as e:
        print(f"❌ ファインチューニング中にエラー: {e}")
        return None, None

# 訓練実行
if train_df is not None and peft_model is not None:
    trainer, label_encoder = train_model(train_df, peft_model, tokenizer, device)
else:
    print("❌ 訓練に必要な要素が準備されていません")
    trainer, label_encoder = None, None

## 💾 モデル保存

In [ ]:
def save_trained_model(trainer, tokenizer, label_encoder):
    """訓練済みモデルの保存（Kaggle提出用形式）"""
    print("\n" + "=" * 60)
    print("💾 訓練済みモデル保存")
    print("=" * 60)

    if trainer is None or tokenizer is None or label_encoder is None:
        print("❌ 保存に必要な要素が準備されていません")
        return False

    save_base_path = "/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models"
    model_save_path = f"{save_base_path}/gemma-2-2b-math-classification-final"

    try:
        # モデルとトークナイザーの保存
        print(f"📁 保存先: {model_save_path}")
        
        # トレーナーからモデルを保存（PEFTモデル込み）
        trainer.save_model(model_save_path)
        tokenizer.save_pretrained(model_save_path)
        
        print("✅ モデルとトークナイザー保存完了")

        # ラベルマッピングの保存
        label_mapping = {i: label for i, label in enumerate(label_encoder.classes_)}
        label_file = f"{model_save_path}/label_mapping.json"
        
        with open(label_file, "w", encoding="utf-8") as f:
            json.dump(label_mapping, f, ensure_ascii=False, indent=2)
        
        print("✅ ラベルマッピング保存完了")
        print(f"📋 ラベルマッピング: {label_mapping}")

        # メタデータの保存
        metadata = {
            "model_name": MODEL_NAME,
            "num_labels": NUM_LABELS,
            "max_length": MAX_LENGTH,
            "task_type": "sequence_classification",
            "architecture": "AutoModelForSequenceClassification",
            "peft_applied": True,
            "lora_config": {
                "r": 16,
                "lora_alpha": 32,
                "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
                "task_type": "SEQ_CLS"
            }
        }
        
        metadata_file = f"{model_save_path}/model_metadata.json"
        with open(metadata_file, "w", encoding="utf-8") as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        
        print("✅ メタデータ保存完了")

        # 保存されたファイルの確認
        print(f"\n📂 保存されたファイル:")
        for item in os.listdir(model_save_path):
            item_path = os.path.join(model_save_path, item)
            if os.path.isfile(item_path):
                size_mb = os.path.getsize(item_path) / (1024 * 1024)
                print(f"  📄 {item}: {size_mb:.2f} MB")
            else:
                print(f"  📁 {item}/")

        return True

    except Exception as e:
        print(f"❌ モデル保存中にエラー: {e}")
        return False

# モデル保存実行
if trainer is not None and tokenizer is not None and label_encoder is not None:
    save_success = save_trained_model(trainer, tokenizer, label_encoder)
    
    if save_success:
        print("\n🎉 モデル保存完了!")
        print("📁 Google Drive保存パス: /content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/")
        print("🚀 次のステップ: Kaggleデータセットとしてアップロード準備完了")
    else:
        print("❌ モデル保存に失敗しました")
else:
    print("❌ 保存に必要な要素が準備されていません")

## 📦 ローカルダウンロード用zip作成

In [ ]:
def create_download_zip():
    """ローカルダウンロード用のzipファイル作成"""
    print("\n" + "=" * 60)
    print("📦 ローカルダウンロード用zip作成")
    print("=" * 60)

    try:
        import zipfile
        
        model_dir = "/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/gemma-2-2b-math-classification-final"
        zip_path = "/content/drive/MyDrive/kaggleCompe_MAP-math/gemma-2-2b-math-model.zip"
        
        if not os.path.exists(model_dir):
            print(f"❌ モデルディレクトリが見つかりません: {model_dir}")
            return False
        
        print(f"📁 ソース: {model_dir}")
        print(f"📦 出力zip: {zip_path}")
        
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, dirs, files in os.walk(model_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, model_dir)
                    zipf.write(file_path, arcname)
                    print(f"  ➕ {arcname}")
        
        # zipファイルサイズ確認
        zip_size_mb = os.path.getsize(zip_path) / (1024 * 1024)
        print(f"\n✅ zip作成完了!")
        print(f"📏 zipサイズ: {zip_size_mb:.2f} MB")
        print(f"📥 ダウンロードパス: {zip_path}")
        
        # ダウンロード用コード表示
        print(f"\n💡 Colabでダウンロードするには:")
        print(f"```python")
        print(f"from google.colab import files")
        print(f"files.download('{zip_path}')")
        print(f"```")
        
        return True

    except Exception as e:
        print(f"❌ zip作成中にエラー: {e}")
        return False

# zip作成実行
zip_success = create_download_zip()

## 🎯 完了サマリー

### ✅ 完了タスク
1. **🤖 モデル読み込み**: Gemma-2-2b-it (AutoModelForSequenceClassification)
2. **🔧 LoRA適用**: 効率的なファインチューニング
3. **📊 データ準備**: 6分類タスク用前処理
4. **🚀 訓練実行**: MAP@3最適化
5. **💾 モデル保存**: Google Drive + zip形式

### 📁 保存場所
- **Google Drive**: `/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/`
- **zip形式**: `/content/drive/MyDrive/kaggleCompe_MAP-math/gemma-2-2b-math-model.zip`

### 🔄 次のステップ
1. **Kaggleデータセット作成**: 保存されたモデルをKaggleにアップロード
2. **推論ノートブック**: test_saved2-2b.ipynbで読み込みテスト
3. **提出**: submission.csvの生成

### 🎯 Kaggle提出用読み込み形式
```python
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
```

**🎉 Gemma-2-2b-it分類モデル訓練完了!**

In [ ]:
# 最終実行確認とダウンロード
if zip_success:
    print("🎉 すべての処理が完了しました!")
    print("\n📥 モデルをダウンロードするには以下を実行:")
    print("```python")
    print("from google.colab import files")
    print("files.download('/content/drive/MyDrive/kaggleCompe_MAP-math/gemma-2-2b-math-model.zip')")
    print("```")
    
    # 自動ダウンロード（オプション）
    try:
        from google.colab import files
        response = input("今すぐダウンロードしますか？ (y/n): ")
        if response.lower() == 'y':
            files.download('/content/drive/MyDrive/kaggleCompe_MAP-math/gemma-2-2b-math-model.zip')
            print("✅ ダウンロード開始!")
    except:
        print("💡 手動でダウンロードしてください")
else:
    print("❌ 一部の処理が完了していません。上記のセルを確認してください。")